In [ ]:
import pyfits,glob,time,scipy,re,time
import scipy.interpolate
from numpy import *
import numpy
from matplotlib.pyplot import *
import matplotlib.pyplot as plt
from subprocess import call
import ipyparallel
from itertools import cycle
from scipy import misc
from scipy.ndimage import shift
from scipy import stats

matplotlib.rc('xtick', labelsize=15) 
matplotlib.rc('ytick', labelsize=15)
matplotlib.rc('font',size=30)

import pylab
pylab.rcParams['font.size']=30
topdir='/Users/cslage/Research/LSST/code/GUI/'

%matplotlib inline

In [ ]:
def mosaic_combine_ITL(filename):
    "Takes in a multi-extension mosaic fits file and the dimensions of the mosaic and returns an array with all chips combined"
    dim_x,dim_y=8,2
    hdr0 = pyfits.getheader(filename,0)
    numccds = hdr0['nextend']
    sizex,sizey = hdr0['IMG_COLS'],hdr0['IMG_ROWS']
    overscanx,overscany = hdr0['OVR_COLS'],hdr0['OVR_ROWS']
    prescanx,prescany = hdr0['PRE_COLS'],hdr0['PRE_ROWS']
    sizex,sizey = sizex,sizey
    
    ccdarray = np.zeros((sizey*dim_y,sizex*dim_x),dtype='float')
        
    for ccdnum in range(numccds):
        ccdseg = pyfits.getdata(filename,ccdnum+1)
        med_overscan=median(ccdseg[:,-overscanx:])
        ccdseg=ccdseg[prescany:-overscany,prescanx:-overscanx]-med_overscan
        j=ccdnum/dim_x   # depends on int division
        if ccdnum<dim_x:
            i=dim_x-ccdnum-1
            ccdseg=ccdseg[:,::-1]
        else:
            i=ccdnum-dim_x
            ccdseg=ccdseg[::-1,:]
        ccdarray[j*sizey:(j+1)*sizey,i*sizex:(i+1)*sizex]=ccdseg
    ccdbottom=ccdarray[0:sizey*dim_y/2,0:sizex*dim_x]
    ccdarray[0:sizey*dim_y/2,0:sizex*dim_x]=ccdbottom[:,::-1]
    return ccdarray

def make_whisker_array2(cat,ndiv,r_out,sigma_smooth,):
    """Takes in a sextractor catalog of shapes and makes a whisker plot. Needs the catalog (in LDAC format),
    the number of divisions to cut the image into and the maximum distance to consider a galaxy to smooth
    inside a division, and the smoothing fwhm in pixels"""
    xs,ys=4072,4000
    xg=cat.field('XWIN_IMAGE')
    yg=cat.field('YWIN_IMAGE')
    thetag=cat.field('THETA_IMAGE')
    ag=cat.field('A_IMAGE')
    bg=cat.field('B_IMAGE')

    r=bg/ag
    eg = (1.-r)/(1.+r)
    #de=np.sqrt(x2err**2+y2err**2)
    #e1,e2 = e*np.cos(2.*theta),e*np.sin(2.*theta)
    
    #First size up the field, and then break it up into pieces for individual analysis
    dx = xs/ndiv
    dy = ys/ndiv
    
    #initialize some arrays to hold the subset calculations
    xc,yc = numpy.zeros([ndiv,ndiv]),numpy.zeros([ndiv,ndiv])
    angsub,arrow_len = numpy.zeros([ndiv,ndiv]),numpy.zeros([ndiv,ndiv])
    ngal = numpy.zeros([ndiv,ndiv])
    agrid= numpy.zeros([ndiv,ndiv])
    bgrid= numpy.zeros([ndiv,ndiv])
    
    #go division by division
    for i in range(ndiv):
        for j in range(ndiv):
            # find the center of the division
            xc[i,j] = ((i*1./ndiv)*xs + dx/2)
            yc[i,j] = ((j*1./ndiv)*ys + dy/2)
            #grab the good elements that are inside the subdivision
            sub = numpy.where((abs(xg-xc[i,j])<r_out) & (abs(yg-yc[i,j])<r_out))
            ngal[i,j] = len(sub[0])   #the number of galaxies used in the calc
            #calculate each element's distance from the center
            dx2 = (xc[i,j]-xg[sub])**2
            dy2 = (yc[i,j]-yg[sub])**2
            r2 = dx2 + dy2
            #and then apply the radius from center in a gaussian weight
            weight = 1.*numpy.exp(-.5*r2/sigma_smooth**2)#/degsub**2
            weightsum = numpy.sum(weight)
            weight=weight/weightsum
            agrid[i,j] = numpy.sum(ag[sub]*weight)
            bgrid[i,j] = numpy.sum(bg[sub]*weight)
            scalar_ellip = numpy.sum(eg[sub]*weight)#numpy.sqrt(e1sub[i,j]**2+e2sub[i,j]**2)
            arrow_len[i,j] = scalar_ellip*xs
            angsub[i,j] = numpy.sum(thetag[sub]*weight)
            #phi = .5*numpy.arctan2(e2sub[i,j],e1sub[i,j])
    return xc,yc,arrow_len,angsub,ngal,agrid,bgrid

def run_sextractor_on_mosaic(fitsfilename):
    #from mydefs import make_ell_reg_from_xy
    #from mydefs import make_reg_from_ldac
    import pyfits,numpy
    from subprocess import call

    configfile=topdir+'sextractor/30-micron-pinholes.sex'
    #configfile=topdir+'sextractor/300-micron-pinholes.sex'
    paramfile=topdir+'sextractor/default-array_dither.param'

    #configfile='/home/andrew/Work/ccd/30-micron-pinhole.sex'
    #paramfile='/home/andrew/Work/ccd/default-array_dither.param'
    #hdr0=pyfits.open(fitsfilename)[0].header
    #pdiode=hdr0['MONDIODE']
    #exptime=hdr0['EXPTIME']
    #fluxval=float(pdiode)*float(exptime)
    minadu=10#5e10*fluxval
    outname=fitsfilename
    callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
               "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
               "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
               "-DETECT_THRESH","10","-DETECT_MINAREA","10","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","400",
               "-ANALYSIS_THRESH","2"]
    #callstring=["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat',"-CATALOG_TYPE","FITS_LDAC",
    #           "-CHECKIMAGE_NAME",outname[:-4]+"back.fits"+','+outname[:-4]+"rms.fits",
    #           "-CHECKIMAGE_TYPE","BACKGROUND,BACKGROUND_RMS",
    #           "-DETECT_THRESH","10","-DETECT_MINAREA","1000","-THRESH_TYPE","RELATIVE","-DETECT_MAXAREA","40000",
    #           "-ANALYSIS_THRESH","2"]
    test=call(callstring)
    #make_ell_reg_from_xy(fitsfilename)
    #make_ell_reg_from_xy(outname)
    #call(["rm",outname])
    #print callstring



def filtcat_mediansig(cat,parname,nsig):
    """Filters a sextractor catalog python object by sigma clipping parname in cat by signum,
    returns the max,min and std deviation of vals"""
    parvals=cat[parname]
    parvals=parvals#[where(cat['FLAGS']==0)]
    medparval=median(parvals)
    stdparval=std(parvals)
    maxparval,minparval=medparval+nsig*stdparval,medparval-nsig*stdparval
    parvals=parvals[where((parvals<maxparval) & (parvals>minparval))]
    medparval=median(parvals)
    stdparval=std(parvals)
    maxparval,minparval=medparval+nsig*stdparval,medparval-nsig*stdparval
    goodobjs=where((cat[parname]<maxparval) & (cat[parname]>minparval))# & (cat['FLAGS']==0))
    return cat[goodobjs],minparval,maxparval,stdparval



In [ ]:
def make_ell_reg_from_xy(filename):
    """Takes in X & Y coordinates and a text tag and outputs a DS9 region file"""
    foocat=pyfits.getdata(filename+'.cat',"LDAC_OBJECTS")
    xreg,yreg=foocat['XWIN_IMAGE'],foocat['YWIN_IMAGE']
    radius1=7*np.ones(len(xreg))
    radius2=foocat['B_IMAGE']/foocat['A_IMAGE']*radius1
    anglereg=foocat['THETA_IMAGE']
    text_tag=['']*len(xreg)
    reg_filename=filename+'.reg'
    f = open(reg_filename,'w')
    f.write('image'+' \n')
    for i in range(len(xreg)):
        thetext=text_tag[i]
        f.write('ellipse '+str(xreg[i])+' '+str(yreg[i])+' '+str(radius1[i])+' '+str(radius2[i])+
                ' '+str(anglereg[i])+' #text="'+str(thetext)+'"\n')
    f.close()

In [ ]:
thedir='/Users/cslage/Research/LSST/code/GUI/cte/galsim/bf_9/'
expnum_root,date='5','20170313'
filelist=sort(glob.glob(thedir+'bf_1.fits'))
print len(filelist)," files"
zpos_all=[]
for filename in filelist:
    zpos=0.0#pyfits.getheader(filename,17)['Y_POS']
    zpos_all.append(zpos)
    #print '#',filename.split('/')[-1].split('_')[3],zpos

In [ ]:
for filename in filelist:
    for charge_transfer in [1000,800,600,400,200,100, 30, 10, 3, 1]:
        newfile = filename[:-5]+'_CTI_%04d.fits'%charge_transfer
        data = pyfits.getdata(filename,0)
        print charge_transfer, data.max()
        for i in range(data.shape[0]):
            for j in range(data.shape[1]-1):
                data[i,j] = data[i,j] + (data[i,j+1] * charge_transfer) / 10000
        print charge_transfer, data.max()
        pyfits.writeto(newfile, data ,header=pyfits.getheader(filename,0),clobber='True')





In [ ]:
mosfilelist=sort(glob.glob(thedir+'bf_1.fits')+glob.glob(thedir+'bf_1_CTI_????.fits'))
print len(mosfilelist)

In [ ]:
tstart=time.time()
for i in range(len(mosfilelist)): run_sextractor_on_mosaic(mosfilelist[i])

print time.time()-tstart

In [ ]:
print mosfilelist

In [ ]:
# Now you should be able to check out the fits images and region files together.
# to open the first one (which is usually a good check) open the .fits and .reg like:
# ds9 *_800_*whole.fits -regions *_800_*.reg

In [ ]:
figure(figsize=(16,8))
suptitle('Ellipticity Phase Angles', fontsize = 24)
emags_mean = []
emags_std = []
eangs_mean = []
eangs_std = []
cts = []
plotnums = [0,4,6,8]
plotcounter = 1
for filenum, mosfilename in enumerate(mosfilelist):
    if filenum == 0:
        ct = 1.0E-7 / 512.0 * 1.0E5
    else:
        ct = int(mosfilename.split('/')[-1].split('_')[-1].strip('.fits')) / 10000.0 / 512.0 * 1.0E5
    cts.append(ct)        
    catname=mosfilelist[filenum]+'.cat'
    cat=pyfits.getdata(catname,"LDAC_OBJECTS")
    x2 = cat['X2WIN_IMAGE']
    y2 = cat['Y2WIN_IMAGE']
    xy = cat['XYWIN_IMAGE']
    denom = (x2+y2+2.0*sqrt(x2*y2-xy*xy))
    er = (x2 - y2) / denom
    ei = (2.0 * xy) / denom
    emag = sqrt(er * er + ei * ei)
    emag_mean = sqrt(mean(er)*mean(er) + mean(ei)*mean(ei))
    emags_mean.append(emag_mean)
    eangs_mean.append(arctan2(mean(ei),mean(er)) * 180.0 / pi)
    emag_std = sqrt(er * er * std(er) * std(er) + ei * ei * std(ei) * std(ei)) / emag
    emags_std.append(std(emag_std) / sqrt(len(emag_std)))
    eang_std = sqrt(ei * ei * std(er) * std(er) + er * er * std(ei) * std(ei)) / (emag * emag)
    eangs_std.append(std(eang_std) * 90.0 / pi / sqrt(len(emag_std)))
    if filenum in plotnums:
        subplot(2,2,plotcounter)
        title("CTI = %.1f * 10^-5"%ct)
        bins=90
        hist(arctan2(ei,er)*90.0 / pi, bins=bins)
        xlabel("Angle(Degrees)", fontsize = 12)
        plotcounter += 1
subplots_adjust(hspace = 0.5, wspace = 0.5)    
savefig(thedir+"Ellipse_Angles_6Apr18.png")



figure(figsize=(16,8))
suptitle('CTI Impact on Complex Ellipticity', fontsize = 24)
subplot(1,2,1)
title('Magnitude of Complex Ellipticity', fontsize = 18)
#plot(cts, emags_mean, marker = '.', ms = 10)
errorbar(cts, emags_mean, yerr=emags_std, marker = '.', ms = 10)
#errorbar(cts, ell_mean, yerr=ell_std, marker = '.', ms = 10)
plot([5.0,5.0],[0.0,0.01],ls='--',color='red')
xlim(0.0, 10.0)
xticks([0.0,2.0,4.0,6.0,8.0,10.0])
ylim(0.0, .004)
ylabel('Ellipticity', fontsize=12)    
xlabel('EPER CTI (*1E-5)', fontsize=12)    

subplot(1,2,2)
title('Phase Angle of Complex Ellipticity', fontsize = 18)
#plot(cts, eangs_mean, marker = '.', ms = 10)
errorbar(cts, eangs_mean, yerr=eangs_std, marker = '.', ms = 10)
#errorbar(cts, th_mean, yerr=th_std, marker = '.', ms = 10)
plot([5.0,5.0],[-60.0,60.0],ls='--',color='red')
xlim(0.0, 10.0)
xticks([0.0,2.0,4.0,6.0,8.0,10.0])
ylim(-60, 60)
ylabel('Angle (Degrees)', fontsize=12)       
xlabel('EPER CTI (*1E-5)', fontsize=12)       

subplots_adjust(wspace = 0.5)
    
savefig(thedir+"Ellipticity_vs_CTI_24Apr18.png")



In [ ]:
figure(figsize=(16,8))
suptitle('Ellipticity Phase Angles', fontsize = 24)
emag_mean = []
emag_std = []
eang_mean = []
eang_std = []
cts = []
plotnums = [0,4,6,8]
plotcounter = 1
for filenum, mosfilename in enumerate(mosfilelist):
    if filenum == 0:
        ct = 1.0E-7 / 512.0 * 1.0E5
    else:
        ct = int(mosfilename.split('/')[-1].split('_')[-1].strip('.fits')) / 10000.0 / 512.0 * 1.0E5
    cts.append(ct)        
    catname=mosfilelist[filenum]+'.cat'
    cat=pyfits.getdata(catname,"LDAC_OBJECTS")
    x2 = cat['X2WIN_IMAGE']
    y2 = cat['Y2WIN_IMAGE']
    xy = cat['XYWIN_IMAGE']
    a = cat['A_IMAGE']
    b = cat['B_IMAGE']
    t = cat['THETA_IMAGE']
    denom = (x2+y2+2.0*sqrt(x2*y2-xy*xy))
    er = (x2 - y2) / denom
    ei = (2.0 * xy) / denom
    emag_mean.append(sqrt(mean(er)*mean(er) + mean(ei)*mean(ei)))
    eang_mean.append(arctan2(mean(ei),mean(er)) * 180.0 / pi)
    if filenum in plotnums:
        subplot(2,2,plotcounter)
        title("CTI = %.1f * 10^-5"%ct)
        bins = 180
        #hist(arctan2(ei,er)*90.0 / pi,bins=bins)
        #hist(t,bins=bins)
        #scatter(arctan2(ei,er)*90.0 / pi, t, s = 0.1)
        scatter(t, xy, color='red', s = 0.2, label = 'Sex THETA')
        scatter(arctan2(ei,er)*90.0 / pi, xy, color='blue', s = 0.2, label = 'arctan2(ei,er)')

        ylim(-.005, .005)
        #xlabel("Arctan2 Theta", fontsize = 12)
        #xlabel("Sextractor Theta", fontsize = 12)
        xlabel("Theta", fontsize = 12)
        ylabel("Sextractor XY")
        legend(loc = 'lower right')
        plotcounter += 1
subplots_adjust(hspace = 0.5, wspace = 0.5)    
#savefig(thedir+"Ellipse_Hist1_6Apr18.png")
savefig(thedir+"Ellipse_Angles_6Apr18.png")

